In [1]:
import warnings
warnings.filterwarnings('ignore')
import pygad

# Bibliotecas principais do TensorFlow e Keras para Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from scikeras.wrappers import KerasRegressor


# Manipulação e análise de dados
import pandas as pd
import numpy as np

# Visualização de dados
import matplotlib.pyplot as plt

# Controle de aleatoriedade e ambiente
import random
import os

# Pré-processamento e divisão de dados
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Métricas de avaliação
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression

# Bibliotecas auxiliares
import aux_functions as aux
import importlib
importlib.reload(aux)

2025-06-02 13:16:07.841543: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-02 13:16:07.948114: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748880967.982219 2883437 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748880967.991409 2883437 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-02 13:16:08.078632: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

<module 'aux_functions' from '/home/elizeta/projects/Iniciação Científica/Otimizacao-de-Gas-com-Neuro-Evolucao/Trends_Atmospheric_CO2/aux_functions.py'>

In [2]:
usar_GPU = False #Deixar True ou False

if usar_GPU:
    print("Dispositivos visíveis:", tf.config.get_visible_devices())
else:
    tf.config.set_visible_devices([], 'GPU')
    print("Dispositivos visíveis:", tf.config.get_visible_devices())
    tf.keras.mixed_precision.set_global_policy('mixed_float16')

Dispositivos visíveis: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [3]:
series = aux.ler_csv('dataSet.csv', average_only=True)

In [4]:
dataset = series.values
dataset = dataset.astype(np.float32)

In [5]:
window_size = 12
X, y = aux.criarJanelas(dataset, window_size)

In [6]:
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.15)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Criar modelo
def build_model(neurons, dense_layers, optimizer_code):
    model = Sequential()
    model.add(Input(shape=(X_train_scaled.shape[1],)))
    for _ in range(dense_layers):
        model.add(Dense(neurons, activation='relu'))
    model.add(Dense(neurons, activation='relu'))
    model.add(Dense(1, activation='linear'))
    optimizer = Adam() if optimizer_code == 0 else RMSprop()
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

# Função de avaliação
def fitness_func(ga_instance, solution, sol_idx):
    try:
        neurons = int(solution[0])
        dense_layers = int(solution[1])
        optimizer_code = int(solution[2])
        batch_size = int(solution[3])
        epochs = int(solution[4])

        print(f"[FITNESS] Neurons={neurons}, Layers={dense_layers}, Opt={'Adam' if optimizer_code==0 else 'RMSprop'}, Batch={batch_size}, Epochs={epochs}")

        model = build_model(neurons, dense_layers, optimizer_code)
        model.fit(X_train_scaled, 
                  y_train,
                  validation_data=(X_val_scaled, y_val),
                  epochs=epochs, 
                  batch_size=batch_size, 
                  verbose=0
                  )

        preds = model.predict(X_val_scaled, verbose=0)
        mse = mean_squared_error(y_val, preds)
        return 1.0 / (mse + 1e-8)
    except Exception as e:
        print(f"[ERRO NA FITNESS] {e}")
        return 0

# Espaço de busca
gene_space = [
    {'low': 8, 'high': 64},     # neurônios
    {'low': 1, 'high': 4},       # número de camadas densas
    {'low': 0, 'high': 1},       # otimizador (0=Adam, 1=RMSprop)
    {'low': 8, 'high': 64},      # batch size
    {'low': 200, 'high': 1000}     # épocas
]

# Instância do GA
ga_instance = pygad.GA(
    num_generations=10,
    num_parents_mating=4,
    fitness_func=fitness_func,
    sol_per_pop=10,
    num_genes=len(gene_space),
    gene_space=gene_space,
    parent_selection_type="rank",
    crossover_type="uniform",
    mutation_type="random",
    mutation_percent_genes=20,
    on_generation=lambda ga: print(f"Ger {ga.generations_completed} - Melhor fitness: {ga.best_solution()[1]:.6f}")
)

ga_instance.run()

# Melhor solução
solution, fitness, _ = ga_instance.best_solution()
print("\nMelhor conjunto de hiperparâmetros:")
print(f"Neurônios: {int(solution[0])}")
print(f"Camadas densas: {int(solution[1])}")
print(f"Otimizador: {'Adam' if int(solution[2]) == 0 else 'RMSprop'}")
print(f"Batch size: {int(solution[3])}")
print(f"Épocas: {int(solution[4])}")
print(f"Fitness: {fitness}")


[FITNESS] Neurons=55, Layers=3, Opt=Adam, Batch=59, Epochs=299


I0000 00:00:1748880970.550450 2884672 service.cc:148] XLA service 0x7c9e8000d870 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748880970.550566 2884672 service.cc:156]   StreamExecutor device (0): Host, Default Version
2025-06-02 13:16:10.578051: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1748880970.962822 2884672 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-06-02 13:16:23.722102: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


[FITNESS] Neurons=46, Layers=3, Opt=Adam, Batch=12, Epochs=577
[FITNESS] Neurons=25, Layers=3, Opt=Adam, Batch=59, Epochs=369
[FITNESS] Neurons=46, Layers=3, Opt=Adam, Batch=49, Epochs=413
[FITNESS] Neurons=28, Layers=3, Opt=Adam, Batch=36, Epochs=953
[FITNESS] Neurons=36, Layers=3, Opt=Adam, Batch=31, Epochs=610
[FITNESS] Neurons=14, Layers=3, Opt=Adam, Batch=59, Epochs=790
[FITNESS] Neurons=14, Layers=2, Opt=Adam, Batch=19, Epochs=886
[FITNESS] Neurons=55, Layers=1, Opt=Adam, Batch=27, Epochs=280
[FITNESS] Neurons=31, Layers=1, Opt=Adam, Batch=14, Epochs=730
[FITNESS] Neurons=55, Layers=3, Opt=Adam, Batch=58, Epochs=299
[FITNESS] Neurons=31, Layers=2, Opt=Adam, Batch=14, Epochs=730
[FITNESS] Neurons=31, Layers=2, Opt=Adam, Batch=14, Epochs=577
[FITNESS] Neurons=46, Layers=3, Opt=Adam, Batch=12, Epochs=369
[FITNESS] Neurons=55, Layers=3, Opt=Adam, Batch=59, Epochs=369
[FITNESS] Neurons=31, Layers=3, Opt=Adam, Batch=59, Epochs=299
[FITNESS] Neurons=31, Layers=1, Opt=Adam, Batch=12, Epo